In [1]:
import time
import pickle
import pathlib
from scipy.stats import entropy
import numpy as np 
from solver.policy.random_policy import RandomFinitePolicy
from solver.stoc_reg_omd_graphon_solver import stoc_reg_DiscretizedGraphonExactOMDSolverFinite
from solver.stoc_reg_omd_graphon_solver_2 import stoc_reg_DiscretizedGraphonExactOMDSolverFinite_2
from solver.reg_omd_graphon_solver import reg_DiscretizedGraphonExactOMDSolverFinite
from solver.omd_graphon_solver import DiscretizedGraphonExactOMDSolverFinite
from solver.reg_graphon_solver import reg_DiscretizedGraphonExactSolverFinite
from solver.graphon_solver import DiscretizedGraphonExactSolverFinite
from evaluator.graphon_evaluator import DiscretizedGraphonEvaluatorFinite
from evaluator.reg_graphon_evaluator import reg_DiscretizedGraphonEvaluatorFinite
from evaluator.stochastic_evaluator import StochasticEvaluator
from games.finite.beach import BeachGraphon
from games.finite.cyber import CyberGraphon
from games.finite.cyber_het import HeterogeneousCyberGraphon
from games.finite.investment import InvestmentGraphon
from games.finite.sis import SISGraphon
from games.graphons import uniform_attachment_graphon, er_graphon, ranked_attachment_graphon, power_law_graphon, cutoff_power_law_graphon, sbm_graphon, exp_graphon
from simulator.graphon_simulator import DiscretizedGraphonExactSimulatorFinite
from simulator.stochastic_simulator import StochasticSimulator
""" Initialize """

' Initialize '

In [3]:
def const_graphon_1(x,y):
    return 1
def const_graphon_2(x,y):
    return 0.5
def const_graphon_3(x,y):
    return 0

regularization = 1 
num_alphas = 30
game = BeachGraphon(**{"graphon": exp_graphon})
# game1 = BeachGraphon(**{"graphon": const_graphon_1})
# game2 = BeachGraphon(**{"graphon": const_graphon_2})
# game3 = BeachGraphon(**{"graphon": const_graphon_3})
simulator = DiscretizedGraphonExactSimulatorFinite(**{})
evaluator = reg_DiscretizedGraphonEvaluatorFinite(**{'regularization':regularization,'num_alphas':num_alphas})
#solver = stoc_reg_DiscretizedGraphonExactOMDSolverFinite(**{'regularization':regularization, 'num_alphas': num_alphas, 'K': 10})
solver1 = stoc_reg_DiscretizedGraphonExactOMDSolverFinite_2(**{'regularization':regularization, 'num_alphas': 5, 'K': 300})
# solver2 = stoc_reg_DiscretizedGraphonExactOMDSolverFinite_2(**{'regularization':regularization, 'num_alphas': 10, 'K': 100})
solver3 = reg_DiscretizedGraphonExactOMDSolverFinite(**{'regularization':regularization,'num_alphas':num_alphas})
# solver4 = DiscretizedGraphonExactOMDSolverFinite(**{'num_alphas':num_alphas})

eval_solver = reg_DiscretizedGraphonExactSolverFinite(**{'regularization':regularization,'num_alphas':num_alphas})
iterations = 25

big_logs = {}

In [5]:
policy = RandomFinitePolicy(game.agent_observation_space, game.agent_action_space)
mu, info = simulator.simulate(game, policy)


In [21]:
# game.reward_g(t=0,x=[0.2,5],u=1,g=policy)
mu.mu_alphas[100].mus[10][5]
alphas  = mu.alphas
a = 0.03
index = np.argmin(abs(alphas-a))


3

In [72]:
from games.mfg_wrapper import MFGGymWrapper
data_cache = []
env = MFGGymWrapper(game, mu, time_obs_augment=False)
temp_policy = RandomFinitePolicy(game.agent_observation_space, game.agent_action_space)
for alpha in solver1.alphas:
    alpha_data_log = []
    for _ in range(solver1.K):
        temp_data_log = []
        observation = env.reset()
        observation = (alpha,observation[1])
        done = 0
        while not done:
            action = temp_policy.act(env.t, observation)
            next_observation, reward, done, _ = env.step(action)
            temp_data_log.append([observation,action,reward,next_observation])
            observation = next_observation

        alpha_data_log.append(temp_data_log)
    data_cache.append(alpha_data_log)
    #print("finish agent initialization")

Q_alphas = []

In [73]:
for agent_idx,alpha in zip(range(len(solver1.alphas)), solver1.alphas):
            Vs = []
            Qs = []
            curr_V = [0 for _ in range(game.agent_observation_space[1].n)]

            for t in range(game.time_steps).__reversed__():
                Q_t_pi = []
                for x in range(game.agent_observation_space[1].n):
                    x = tuple([alpha, x])

                    Q_tx_pi = []
                    temp_data = [data_cache[agent_idx][repeat][t] for repeat in range(solver1.K)]
                    for u in range(game.agent_action_space.n):
                        transit_est = []
                        temp_samples = []
                        for index in range(solver1.K):
                            if temp_data[index][0][1]== x[1] and temp_data[index][1]== u:
                                temp_samples.append(temp_data[index][3][1])
                        #print(temp_samples[0])
                        if len(temp_samples):
                            for temp_S in range(game.agent_observation_space[1].n):
                                temp_count = temp_samples.count(temp_S)/len(temp_samples)
                                transit_est.append(temp_count)
                        else:
                            #print(alpha)
                            #print(t)
                            for temp_S in range(game.agent_observation_space[1].n):
                                transit_est.append(1/game.agent_observation_space[1].n)
                        #print(transit_est)
                        temp_Q_est = game.reward(t, x, u, mu) + (1 - game.done(t, x, u, mu)) * np.vdot(curr_V, transit_est)
                        Q_tx_pi.append(temp_Q_est)
                    #           np.vdot(curr_V, mfg.transition_probs(t, x, u, mu))
                    #Q_tx_pi = [mfg.reward(t, x, u, mu) + (1 - mfg.done(t, x, u, mu)) *
                    #           np.vdot(curr_V, mfg.transition_probs(t, x, u, mu))
                    #           for u in range(mfg.agent_action_space.n)]
                    #Q_t_pi is a Q-function at time t 
                    Q_t_pi.append(Q_tx_pi)
                curr_V = [np.vdot(Q_t_pi[x], policy.pmf(t, tuple([alpha, x])))+entropy(policy.pmf(t, tuple([alpha, x]))) for x in range(len(curr_V))]
                Vs.append(curr_V)
                Qs.append(Q_t_pi)
            Vs.reverse()
            Qs.reverse()
            Q_alphas.append(Qs)
if solver1.y is None:
    solver1.y = solver1.omd_coeff2 * np.array(Q_alphas)
else:
    solver1.y = solver1.omd_coeff1*solver1.y + solver1.omd_coeff2 * np.array(Q_alphas)

array([0.07229388, 0.27398899, 0.65371713])

##### Now we can update the policy 

In [77]:
from solver.policy.graphon_policy import DiscretizedGraphonFeedbackPolicy
from solver.policy.finite_policy import QMaxPolicy, QSoftMaxPolicy
policy = DiscretizedGraphonFeedbackPolicy(game.agent_observation_space, game.agent_action_space,
                                                  [
                                                      QSoftMaxPolicy(game.agent_observation_space, game.agent_action_space, Qs, 1 / solver1.eta)
                                                      for Qs, alpha in zip(solver1.y, solver1.alphas)
                                                  ], solver1.alphas)

In [103]:
p = game.transition_probability_matrix(t, alpha, policy,
                                                DiscretizedGraphonMeanField(game.agent_observation_space,
                                                                            [ConstantFiniteMeanField(
                                                                                game.agent_observation_space,
                                                                                mu_alpha)
                                                                            for mu_alpha in mu_alphas_curr],
                                                                            simulator.alphas))

mu_new = DiscretizedGraphonMeanField(game.agent_observation_space,[ConstantFiniteMeanField(game.agent_observation_space,mu_alpha)for mu_alpha in mu_alphas_curr],simulator.alphas)


np.sum([np.array([policy.pmf(t, tuple([alpha, x]))[u] * game.transition_probs(t, tuple([alpha, x]), u, mu_new)
                                 for x in range(game.agent_observation_space[1].n)])
                       for u in range(policy.action_space.n)], axis=0)

[policy.pmf(t, tuple([alpha, x]))[u] * game.transition_probs(t, tuple([alpha, x]), u, mu_new)
                                 for x in range(game.agent_observation_space[1].n)]



array([0.  , 0.05, 0.9 , 0.05, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ])

In [66]:
from simulator.mean_fields.graphon_mf import DiscretizedGraphonMeanField
from simulator.mean_fields.finite_mf import ConstantFiniteMeanField, ExactFiniteMeanField
mus = []
mu_alphas_curr = []
for alpha in simulator.alphas:
    mu_alphas_curr.append(game.initial_state_distribution.dist2(alpha).probs.numpy())

for t in range(game.time_steps):
    mus.append(mu_alphas_curr)
    mu_alphas_next = []
    for idx, alpha in zip(range(len(simulator.alphas)), simulator.alphas):
        p = game.transition_probability_matrix(t, alpha, policy,
                                                DiscretizedGraphonMeanField(game.agent_observation_space,
                                                                            [ConstantFiniteMeanField(
                                                                                game.agent_observation_space,
                                                                                mu_alpha)
                                                                            for mu_alpha in mu_alphas_curr],
                                                                            simulator.alphas))
        mu_alpha = np.matmul(mu_alphas_curr[idx], p)
        mu_alphas_next.append(mu_alpha)

    mu_alphas_curr = mu_alphas_next

""" Reshape """
final_mus = [
    [mus[t][alpha_idx] for t in range(game.time_steps)]
    for alpha_idx in range(simulator.num_alphas)
]

info = {'mus': mus, 'alphas': simulator.alphas}



Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
ERROR: Could not find a version that satisfies the requirement ray (from versions: none)
ERROR: No matching distribution found for ray
